In [7]:
import pandas as pd

# 读取Excel文件
file_path = "/Volumes/Extreme_zzl/MRCP_metadata_filtered_no_processed.xlsx"  # 替换为实际的文件路径
df = pd.read_excel(file_path, engine="openpyxl")

# 确保时间列是 datetime 格式
df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")

# 遍历每个 ID，按照 ID 进行分组计算
for ID, group in df.groupby('ID'):
    # 提取该组中的所有 start_time 和对应的索引
    start_times = group['start_time'].tolist()
    names = group['name'].tolist()  # 获取 name 列，方便查找包含 MRCP 的名称
    indexes = group.index.tolist()

    # 筛选出该组中的 +C 序列，找到 start_time 最小的
    contrast_sequences = group[group["name"].str.contains(r"\+C", case=False)].sort_values(by="start_time")
    
    if not contrast_sequences.empty:
        # 获取首个 +C 序列
        first_contrast_sequence = contrast_sequences.iloc[0]
        
        # 遍历 MRCP 序列，并根据首个 +C 序列的时间判断是 precontrast 还是 postcontrast
        for i, row in group.iterrows():
            if "MRCP" in row["name"]:
                # 初始化最小的 scanduration 值
                min_scan_duration = None
                
                # 计算 MRCP 扫描时间 (通过下一个序列的 start_time)
                current_index = indexes.index(i)
                next_sequence_start_time = start_times[current_index + 1] if current_index + 1 < len(start_times) else None

                # 循环寻找最小的 scanduration > 8 秒 或者找到含 "MRCP" 名称的序列
                while next_sequence_start_time:
                    scan_duration = abs((next_sequence_start_time - row['start_time']).total_seconds())  # 秒
                    
                    if scan_duration > 8:
                        # 找到一个大于 8 的 scanduration，更新最小值
                        min_scan_duration = scan_duration
                        break  # 找到大于 8 秒的值就停止循环

                    # 如果下一个序列名称包含 "MRCP"，则也停止查找
                    next_name = names[current_index + 1] if current_index + 1 < len(names) else None
                    if "MRCP" in next_name:
                        break  # 找到含 MRCP 的序列，停止查找

                    # 如果小于 8，继续往下一个临近序列查找
                    current_index += 1
                    next_sequence_start_time = start_times[current_index + 1] if current_index + 1 < len(start_times) else None

                # 如果找到有效的最小 scanduration
                if min_scan_duration is not None:
                    min_scan_duration = round(min_scan_duration, 2)  # 保留两位小数
                else:
                    min_scan_duration = None

                # 判断 MRCP 是 precontrast 还是 postcontrast
                if row["end_time"] < first_contrast_sequence["start_time"]:
                    df.at[i, "contrast"] = "precontrast"
                    # 计算 time 列 (到 +C 序列的时间差，绝对值)
                    time_diff = abs((first_contrast_sequence["start_time"] - row["end_time"]).total_seconds())  # 秒
                else:
                    df.at[i, "contrast"] = "postcontrast"
                    # 计算 time 列 (到 +C 序列的时间差，绝对值)
                    time_diff = abs((first_contrast_sequence["start_time"] - row["end_time"]).total_seconds())  # 秒

                # 将 time 和 scanduration 结果保留两位小数
                time_diff = round(time_diff, 2)

                # 填充 time 和 scanduration 列
                df.at[i, "scanduration"] = min_scan_duration
                df.at[i, "time"] = time_diff

# 保存结果到新的 Excel 文件
output_path = "/Users/ziling/Desktop/MRCP/temp_output.xlsx"  # 替换为实际的保存路径
df.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到: {output_path}")

处理完成，结果已保存到: /Users/ziling/Desktop/MRCP/temp_output.xlsx


/var/folders/qz/48dx0qtx3w1fc47j_x11r5g00000gn/T/ipykernel_80281/3049297868.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
/var/folders/qz/48dx0qtx3w1fc47j_x11r5g00000gn/T/ipykernel_80281/3049297868.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")


In [2]:
import pandas as pd

# 读取Excel文件
file_path = "/Users/ziling/Desktop/MRCP/data_excel/temp.xlsx"  # 替换为实际的文件路径
df = pd.read_excel(file_path, engine="openpyxl")

# 确保时间列是 datetime 格式
df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")

# 遍历每个 ID，按照 ID 进行分组计算
for ID, group in df.groupby('ID'):
    # 提取该组中的所有 start_time 和对应的索引
    start_times = group['start_time'].tolist()
    names = group['name'].tolist()  # 获取 name 列，方便查找包含 MRCP 的名称
    indexes = group.index.tolist()

    # 筛选出该组中的 +C 序列，找到 start_time 最小的
    contrast_sequences = group[group["name"].str.contains(r"artery", case=False)].sort_values(by="start_time")
    
    if not contrast_sequences.empty:
        # 获取首个 +C 序列
        first_contrast_sequence = contrast_sequences.iloc[0]
        
        # 遍历 MRCP 序列，并根据首个 +C 序列的时间判断是 precontrast 还是 postcontrast
        for i, row in group.iterrows():
            if "MRCP" in row["name"]:
                # 初始化最小的 scanduration 值
                min_scan_duration = None
                
                # 计算 MRCP 扫描时间 (通过下一个序列的 start_time)
                current_index = indexes.index(i)
                next_sequence_start_time = start_times[current_index + 1] if current_index + 1 < len(start_times) else None

                # 循环寻找最小的 scanduration > 8 秒 或者找到含 "MRCP" 名称的序列
                while next_sequence_start_time:
                    scan_duration = abs((next_sequence_start_time - row['start_time']).total_seconds())  # 秒
                    
                    if scan_duration > 8:
                        # 找到一个大于 8 的 scanduration，更新最小值
                        min_scan_duration = scan_duration
                        break  # 找到大于 8 秒的值就停止循环

                    # 如果下一个序列名称包含 "MRCP"，则也停止查找
                    next_name = names[current_index + 1] if current_index + 1 < len(names) else None
                    if "MRCP" in next_name:
                        break  # 找到含 MRCP 的序列，停止查找

                    # 如果小于 8，继续往下一个临近序列查找
                    current_index += 1
                    next_sequence_start_time = start_times[current_index + 1] if current_index + 1 < len(start_times) else None

                # 如果找到有效的最小 scanduration
                if min_scan_duration is not None:
                    min_scan_duration = round(min_scan_duration, 2)  # 保留两位小数
                else:
                    min_scan_duration = None

                # 判断 MRCP 是 precontrast 还是 postcontrast
                if row["end_time"] < first_contrast_sequence["start_time"]:
                    df.at[i, "contrast"] = "precontrast"
                    # 计算 time 列 (到 +C 序列的时间差，绝对值)
                    time_diff = abs((first_contrast_sequence["start_time"] - row["end_time"]).total_seconds())  # 秒
                else:
                    df.at[i, "contrast"] = "postcontrast"
                    # 计算 time 列 (到 +C 序列的时间差，绝对值)
                    time_diff = abs((first_contrast_sequence["start_time"] - row["end_time"]).total_seconds())  # 秒

                # 将 time 和 scanduration 结果保留两位小数
                time_diff = round(time_diff, 2)

                # 填充 time 和 scanduration 列
                df.at[i, "scanduration"] = min_scan_duration
                df.at[i, "time"] = time_diff

# 保存结果到新的 Excel 文件
output_path = "/Users/ziling/Desktop/MRCP/data_excel/temp_output.xlsx"  # 替换为实际的保存路径
df.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到: {output_path}")

/var/folders/qz/48dx0qtx3w1fc47j_x11r5g00000gn/T/ipykernel_32941/2181679111.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["start_time"] = pd.to_datetime(df["start_time"], errors="coerce")
/var/folders/qz/48dx0qtx3w1fc47j_x11r5g00000gn/T/ipykernel_32941/2181679111.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["end_time"] = pd.to_datetime(df["end_time"], errors="coerce")


处理完成，结果已保存到: /Users/ziling/Desktop/MRCP/data_excel/temp_output.xlsx


In [16]:
import pandas as pd

# 读取两个Excel文件
temp_df = pd.read_excel('/Users/ziling/Desktop/MRCP/temp.xlsx')
temp1_df = pd.read_excel('/Users/ziling/Desktop/MRCP/temp1.xlsx')

# 清理 'Old Filename' 列，去除格式问题，如冒号，并转为小写
temp_df['Old Filename_cleaned'] = temp_df['Old Filename'].str.replace(":", "").str.lower()

# 生成用于匹配的 'match_key' 列，结合ID、imgID、start_time，去除冒号并转为小写
temp1_df['match_key'] = temp1_df.apply(lambda row: f"{row['ID']}_{row['imgID']}_{row['start_time']}".replace(":", "").lower() if 'mrcp' in str(row['name']).lower() else None, axis=1)

# 遍历 temp_df 中的每一行，进行匹配并填充 temp1_df 中的 NewFilename 和 Group 列
for i, temp_row in temp_df.iterrows():
    old_filename_cleaned = temp_row['Old Filename_cleaned']
    
    # 在 temp1_df 中查找 match_key 是 Old Filename_cleaned 的子字符串的行
    match_idx = temp1_df[temp1_df['match_key'].apply(lambda x: x in old_filename_cleaned if x else False)].index
    
    # 如果找到了匹配行，填充 NewFilename 和 Group 列
    if not match_idx.empty:
        temp1_df.loc[match_idx, 'NewFilename'] = temp_row['NewFilename']
        temp1_df.loc[match_idx, 'Group'] = temp_row['Group']

# 移除辅助列 'match_key'
temp1_df = temp1_df.drop(columns=['match_key'])

# 直接更新原始的 temp1_df 数据框，不保存到新的文件
print("填充完成。")

填充完成。
